In [9]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sklearn.model_selection as model_selection

import json
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import lightgbm as lgb
import sklearn.model_selection as model_selection
import contextily as cx
import geopandas as gpd

gpd.io.file.fiona.drvsupport.supported_drivers["KML"] = "rw"

In [3]:
# import data 

# Read the apartment datasets
apartments_train = pd.read_csv("resources/data/apartments_train.csv").set_index("id")
apartments_train["split"] = "train"
apartments_test = pd.read_csv("resources/data/apartments_test.csv").set_index("id")
apartments_test["split"] = "test"

# Create a DataFrame of all apartments
apartments = pd.concat([apartments_train, apartments_test])

# Read the building datasets
buildings_train = pd.read_csv("resources/data/buildings_train.csv").set_index("id")
buildings_train["split"] = "train"
buildings_test = pd.read_csv("resources/data/buildings_test.csv").set_index("id")
buildings_test["split"] = "test"

# Create a GeoDataFrame of all buildings
buildings = pd.concat([buildings_train, buildings_test])
buildings = gpd.GeoDataFrame(buildings, geometry=gpd.points_from_xy(
    buildings.longitude, buildings.latitude, crs="EPSG:4326"
))

In [4]:
# Find all buildings missing coordinates
no_coords = buildings.latitude.isna() | buildings.longitude.isna()
street = buildings[~no_coords & (buildings.street == "пос. Коммунарка")]

buildings.loc[no_coords, "latitude"] = street.latitude.mean()
buildings.loc[no_coords, "longitude"] = street.longitude.mean()
buildings.loc[no_coords, "district"] = street.district.mode()[0]

In [5]:
# The coordinates of the southwest and northeast corners of a rectangle approximately encompassing Moscow
MOSCOW_SW_LAT = 55
MOSCOW_SW_LON = 35
MOSCOW_NE_LAT = 57
MOSCOW_NE_LON = 39

In [6]:
# Find all buildings with coordinates outside of Moscow
outside = (((buildings.latitude < MOSCOW_SW_LAT) | (buildings.latitude > MOSCOW_NE_LAT)
          | (buildings.longitude < MOSCOW_SW_LON) | (buildings.longitude > MOSCOW_NE_LON)))
buildings[outside][["split", "latitude", "longitude", "district", "street", "address", "constructed", "material", "stories"]]

for idx, building in buildings[outside].iterrows():
    street = buildings[~outside & (buildings.street == building.street)]
    if len(street):
        buildings.loc[idx, "latitude"] = street.latitude.mean()
        buildings.loc[idx, "longitude"] = street.longitude.mean()
        buildings.loc[idx, "district"] = street.district.mode()[0]
    else:
        buildings.loc[idx, "latitude"] = buildings[~outside].latitude.mean()
        buildings.loc[idx, "longitude"] = buildings[~outside].longitude.mean()

In [7]:
no_district = buildings.district.isna()
districts = buildings.loc[no_district].apply(
    lambda b: buildings.loc[
        (buildings[~no_district][["latitude", "longitude"]] - b[["latitude", "longitude"]]).abs().sum(axis=1).idxmin()
    ].district,
    axis=1
)
districts.rename("district", inplace=True)
buildings.update(districts)

In [10]:
stations = gpd.read_file("resources/metro_stations.kml", driver="KML").drop(columns=["Description"]).rename(columns={"Name": "name"})

# The Earth's radius in meters
EARTH_RADIUS = 6371000

# Create temporary columns for coordinates given in radians
stations["lon_rad"] = np.radians(stations.geometry.x)
stations["lat_rad"] = np.radians(stations.geometry.y)
buildings["lon_rad"] = np.radians(buildings.longitude)
buildings["lat_rad"] = np.radians(buildings.latitude)

# Calculate the distance to the nearest metro station for each building using
# the haversine formula with the Earth's radius as given above
buildings["metro_distance"] = buildings.apply(
    lambda row: np.min(
        2
        * EARTH_RADIUS
        * np.arcsin(
            np.sqrt(
                np.sin((stations.lat_rad - row.lat_rad) / 2) ** 2
                + np.cos(row.lat_rad)
                * np.cos(stations.lat_rad)
                * np.sin((stations.lon_rad - row.lon_rad) / 2) ** 2
            )
        )
    ),
    axis=1,
)


In [11]:
# Read park and garden location data
parks = gpd.read_file("resources/parks_and_gardens.kml", driver="KML").drop(columns=["Description"]).rename(columns={"Name": "name"})

# Create columns for coordinates given in radians
parks["lon_rad"] = np.radians(parks.geometry.x)
parks["lat_rad"] = np.radians(parks.geometry.y)

# Calculate the distance to the nearest park or garden for each building using
# the haversine formula with the Earth's radius as given above
buildings["park_distance"] = buildings.apply(
    lambda row: np.min(
        2
        * EARTH_RADIUS
        * np.arcsin(
            np.sqrt(
                np.sin((parks.lat_rad - row.lat_rad) / 2) ** 2
                + np.cos(row.lat_rad)
                * np.cos(parks.lat_rad)
                * np.sin((parks.lon_rad - row.lon_rad) / 2) ** 2
            )
        )
    ),
    axis=1,
)

# Drop the temporary radian columns
stations.drop(columns=["lon_rad", "lat_rad"], inplace=True)
buildings.drop(columns=["lon_rad", "lat_rad"], inplace=True)

In [12]:
buildings_train = buildings.loc[buildings["split"] == "train", :]
buildings_train.drop(columns = "split", inplace = True)
apartments = pd.read_csv('resources/data/apartments_train.csv')
data = pd.merge(apartments, buildings_train, how='left', left_on='building_id', right_index=True)

buildings_test = buildings.loc[buildings["split"] == "test", :]
buildings_test.drop(columns = "split", inplace = True)
apartments_test = pd.read_csv('resources/data/apartments_test.csv')
data_test = pd.merge(apartments_test, buildings_test, how='left', left_on='building_id', right_index=True)

/Users/jimtotland/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
apartments = pd.read_csv('resources/data/apartments_train.csv')
buildings = pd.read_csv('resources/data/buildings_train.csv')
nan_data = pd.merge(apartments, buildings.set_index('id'), how='left', left_on='building_id', right_index=True)

In [23]:

distance = np.sqrt((37.6 - data["longitude"])**2 + (55.75 - data["latitude"])**2)
data["distance"] = distance
data_train, data_valid = model_selection.train_test_split(data, test_size=0.33, stratify=np.log(data.price).round())

def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5


#input_x = test_data[["area_total", "distance", "rooms", "norm_const", "stories", "floor"]]
#in_dist = test_data[["district"]]

X_train = data_train[["area_total", "longitude", "latitude", "rooms", "constructed", "district", "metro_distance", "park_distance", "distance"]]
y_train = np.log(data_train.loc[X_train.index].price)
X_valid = data_valid[["area_total", "longitude", "latitude", "rooms", "constructed", "district", "metro_distance", "park_distance", "distance"]]
y_valid = np.log(data_valid.loc[X_valid.index].price)

dtrain = xgb.DMatrix(X_train, label = y_train)
dval = xgb.DMatrix(X_valid)
param = {'max_depth': 6, 'eta': 0.2, 'objective': 'reg:squaredlogerror'}
num_round = 2000

bst = xgb.train(param, dtrain, num_round)
preds_train = bst.predict(dtrain)
preds_valid = bst.predict(dval)


print(root_mean_squared_log_error(np.exp(preds_train), np.exp(y_train)))
print(root_mean_squared_log_error(np.exp(preds_valid), np.exp(y_valid)))


0.12573095665058143
0.16190665411486277


In [ ]:
t_apartments = pd.read_csv('resources/data/apartments_test.csv')
t_buildings = pd.read_csv('resources/data/buildings_test.csv')
test_data_a = pd.merge(t_apartments, t_buildings.set_index('id'), how='left', left_on='building_id', right_index=True)
X_test_nan_a = test_data_a.isna().any(axis=1)

In [ ]:
submission = pd.DataFrame()
submission['id'] = test_data_a["id"]
submission.loc[~X_test_nan, 'price_prediction'] = np.exp(numpy_res) # Predict on non-nan entries
submission['price_prediction'].fillna(nn_data["price"].mean(), inplace=True)
print(f'Generated {len(submission)} predictions')

# Export submission to csv with headers
submission.to_csv('xgb_submission.csv', index=False)

# Look at submitted csv
print('\nLine count of submission')
!wc -l sample_submission.csv